In [ ]:
#Section 1: load and preprocess raster (remove 4th band if present)

In [2]:
import rasterio
import numpy as np

# Define the correct raster path
raster_path = [insert source file path]
fixed_raster_path = [insert source file path]
# Open the raster and check the number of bands
with rasterio.open(raster_path) as src:
    print(f" Original raster has {src.count} bands. Expected: 3")

    if src.count > 3:
        print(f" Removing extra bands...")

        # Read only the first 3 bands (Red, Green, Blue)
        data = src.read([1, 2, 3])  # Shape: (3, Height, Width)

        # Update profile for 3-band output
        profile = src.profile
        profile.update(count=3, dtype=data.dtype)

        # Save the new raster
        with rasterio.open(fixed_raster_path, "w", **profile) as dst:
            dst.write(data)

        print(f" Fixed raster saved at: {fixed_raster_path}")
        print(f" Image Shape: {data.shape}")  # Should be (3, Height, Width)
    else:
        print("Raster already has 3 bands. No changes needed.")


📌 Original raster has 4 bands. Expected: 3
📌 Removing extra bands...
✅ Fixed raster saved at: C:\Remote Sensing\Remote Sensing Projects\Fiery Gizzard Trail\Raster Layers\Fixed-Fiery-Gizzard-Trail-5-10-2025-orthophoto.tif
✅ Image Shape: (3, 20293, 29387)


In [ ]:
#Downsample orthmosaic to 10cm/pixel for Deepforest predicitons

In [5]:
import os
import rasterio
from rasterio.enums import Resampling

with rasterio.open(fixed_raster_path) as src:
    scale_factor = 10 / [insert calculated GSD from orthomosaic**]
    new_width = int(src.width / scale_factor)
    new_height = int(src.height / scale_factor)

    data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.bilinear
    )

    new_transform = src.transform * src.transform.scale(
        src.width / new_width,
        src.height / new_height
    )

    profile = src.profile
    profile.update({
        'height': new_height,
        'width': new_width,
        'transform': new_transform
    })
    
    with rasterio.open("downsampled.tif", "w", **profile) as dst:
        dst.write(data)

print("Downsampled raster saved to: downsampled.tif")
downsampled_path = "downsampled.tif"



✅ Downsampled raster saved to: downsampled.tif


In [ ]:
#Section 2: predict tree location using predict_tile().

In [30]:
from deepforest import main

# Initialize DeepForest
model = main.deepforest()
model.load_model(model_name="weecology/deepforest-tree", revision="main")

# Predict on the downsampled image
predicted_results = model.predict_tile(
    downsampled_path, 
    patch_size=400, 
    patch_overlap= 0.35
)

# Print results
print("\n Predictions Before Conversion:")
print(predicted_results.head())


Reading config file: C:\Users\StephenG\AppData\Roaming\Python\Python312\site-packages\deepforest\data\deepforest_config.yml


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Reading config file: C:\Users\StephenG\AppData\Roaming\Python\Python312\site-packages\deepforest\data\deepforest_config.yml


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

16825 predictions in overlapping windows, applying non-max supression
7872 predictions kept after non-max suppression

📌 Predictions Before Conversion:
     xmin    ymin    xmax    ymax label     score       image_path  \
0  5262.0  1829.0  5335.0  1901.0  Tree  0.873955  downsampled.tif   
1  1073.0  2412.0  1146.0  2480.0  Tree  0.862264  downsampled.tif   
2  4695.0  1116.0  4747.0  1171.0  Tree  0.862049  downsampled.tif   
3  3905.0   454.0  3964.0   517.0  Tree  0.859805  downsampled.tif   
4  3405.0  1635.0  3469.0  1700.0  Tree  0.847503  downsampled.tif   

                                            geometry  
0  POLYGON ((5335 1829, 5335 1901, 5262 1901, 526...  
1  POLYGON ((1146 2412, 1146 2480, 1073 2480, 107...  
2  POLYGON ((4747 1116, 4747 1171, 4695 1171, 469...  
3  POLYGON ((3964 454, 3964 517, 3905 517, 3905 4...  
4  POLYGON ((3469 1635, 3469 1700, 3405 1700, 340...  


In [31]:
# Save to working directory
predicted_results.to_csv("Output.csv", index=False)
print("\n Predictions saved to: Output.csv")



✅ Predictions saved to: Output.csv


In [ ]:
#Convert x & y min and max coordinates to geographical coordinatees

In [32]:
from deepforest import utilities
import os

# Define local predictions CSV and raster file in current directory
predictions_path = os.path.abspath("Output.csv")             # Assumes Output.csv is in working directory
raster_path = os.path.abspath("downsampled.tif")             # Local downsampled raster

# Read Predictions
print("\n Reading Predictions CSV...")
df = utilities.read_file(predictions_path)

# Convert to Geographic Coordinates
print("\n Converting to Geographic Coordinates...")
geo_df = utilities.image_to_geo_coordinates(df, root_dir=os.getcwd())

# Print sample to verify conversion
print("\n Sample Converted Predictions:")
print(geo_df.head())




📌 Reading Predictions CSV...

📌 Converting to Geographic Coordinates...

✅ Sample Converted Predictions:
     xmin    ymin    xmax    ymax label     score       image_path  \
0  5262.0  1829.0  5335.0  1901.0  Tree  0.873955  downsampled.tif   
1  1073.0  2412.0  1146.0  2480.0  Tree  0.862264  downsampled.tif   
2  4695.0  1116.0  4747.0  1171.0  Tree  0.862049  downsampled.tif   
3  3905.0   454.0  3964.0   517.0  Tree  0.859805  downsampled.tif   
4  3405.0  1635.0  3469.0  1700.0  Tree  0.847503  downsampled.tif   

                                            geometry  
0  POLYGON ((614032.481 3901573.798, 614032.481 3...  
1  POLYGON ((613651.612 3901520.788, 613651.612 3...  
2  POLYGON ((613979.02 3901638.629, 613979.02 390...  
3  POLYGON ((613907.828 3901698.822, 613907.828 3...  
4  POLYGON ((613862.822 3901591.438, 613862.822 3...  


In [ ]:
#Convert predictions to shapefile

In [33]:
import geopandas as gpd

# Define output path
shapefile_output_path = [insert destination file path]

# Save the converted geographic predictions
geo_df.to_file(shapefile_output_path)

print(f"\n Predictions saved as a shapefile: {shapefile_output_path}")


✅ Predictions saved as a shapefile: C:\Remote Sensing\Remote Sensing Projects\Fiery Gizzard Trail\Deepforest Predictions\Shapefiles\10cm_400patch_0.35overlap.shp
